In [34]:
import pandas as pd
import json


metadata_path = "../../data/single_volumes/dataset.json"
with open(metadata_path, "r") as f:
    dataset_metadata = json.load(f)

table = pd.read_csv("name_map.csv")
name_map:dict[str, str] = pd.Series(table.new_name.values, index=table.old_name).to_dict()
len(name_map)

colors = {
    "nhs": "white",
    "bod": "white",
    "tub": "magenta",
    "cetn": "yellow",
    "dna": "blue",
    "npc": "green",
}
alternative_colors = ["orange", "red"]

FileNotFoundError: [Errno 2] No such file or directory: 'name_map.csv'

In [36]:
counter: int = 0
views = dataset_metadata["views"]
for view_name, view in views.items():
    if view_name == "default":
        continue
    new_name = name_map[view_name]
    if "CHANNEL" in new_name:
        continue
    n_channel_in_view = len(view["sourceDisplays"])
    n_channels_in_name = len(new_name.split("_")[2].split("-"))
    if n_channel_in_view != n_channels_in_name:
        print(view_name, new_name)
        continue
    channels = new_name.split("_")[2].split("-")
    alt_counter: int = 0
    for c in range(n_channel_in_view):
        view["sourceDisplays"][c]["imageDisplay"]["name"] = channels[c]
        channel_name = view["sourceDisplays"][c]["imageDisplay"]["name"]
        # the only case for this is bod-dna-nhs-tub
        # therefore I can just use alternative colors since ornage is the first
        # one there
        if channels[c] == "bod" and "nhs" in channels:
            view["sourceDisplays"][c]["imageDisplay"]["color"] = alternative_colors[
                alt_counter
            ]
            alt_counter += 1
        elif channels[c] in colors.keys():
            view["sourceDisplays"][c]["imageDisplay"]["color"] = colors[channels[c]]
        else:
            view["sourceDisplays"][c]["imageDisplay"]["color"] = alternative_colors[
                alt_counter
            ]
            alt_counter += 1
        if alt_counter > len(alternative_colors):
            print("Ran out of colors")
            print(channels, view_name, new_name)
    counter += 1
dataset_metadata["views"] = views
with open(metadata_path, "w") as f:
    json.dump(dataset_metadata, f, indent=2)
counter

1559